# Imports, checks, paths

In [ ]:
# imports and checks
import pickle
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')


seizurePath = 'gdrive/MyDrive/SeizureCNN/'
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
!pip install mne
import mne
# for the nedc file
!pip install pyedflib
nedc = (seizurePath + 'nedc_pystream.py')
import sys
sys.path.insert(1, seizurePath)
# file from the dataset
from nedc_pystream import *
import mne

In [3]:
# premade splits for training and validation provided by the dataset (for 5-fold cross validation)
validationSplitsPickled = open('gdrive/MyDrive/SeizureCNN/cv_split_5_fold_seizure_wise_v1.5.2.pkl', 'rb')
validationSplits = pickle.load(validationSplitsPickled)

In [4]:
train_path = ''
test_path = '' 
valid_path = '' 

Preprocessing + splitting data 

In [26]:
# to get a sense of what the edf data looks like

file = seizurePath + "00000148_s001_t000.edf"
data = mne.io.read_raw_edf(file)
raw_data = data.get_data()
# you can get the metadata included in the file and a list of all channels:
info = data.info
channels = data.ch_names

Extracting EDF parameters from /content/gdrive/MyDrive/SeizureCNN/00000148_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [ ]:
# this will go in the for loop when we figure out how to split in to smaller epochs
# outputs a list (sample frequencies, signals, labels) each index in the list represents a channel
# signals are the actual data 
fileData = nedc_load_edf(file) 

frequencies = fileData[0]
signals = fileData[1]

In [60]:
# open one specific file  
# using the .tse_bi file because it does not differentiate between different kinds of siezures
with open(seizurePath + "s002_2006_05_18/00002484_s002_t000.tse_bi", "r", encoding='utf-8-sig') as f:
    string = f.read()
# timestamps 
string.splitlines()[2:]

# tse format: start time in secs, stop time in secs, label, probability
# output of above code: 2D list of tse event time stamps 

['0.0000 660.9320 bckg 1.0000',
 '660.9320 800.0480 seiz 1.0000',
 '800.0480 1232.0000 bckg 1.0000']

In [49]:
# for reference: to get the start, and stop times of seizures from the annotations 
def getTimeIndex(time, frequency):
  return time * frequency

-- *code for splitting data into precise epochs will go here*

# Model Definition


In [82]:
# have to determine a good number of channels that is common to all data
#and a protocol for replacing unavailable channels
# 17 channels in commond between all 3 datasets
num_channels = 17
# time stamps per epoch (250 Hz * 10 seconds), most data is sampled at 250 Hz, so 2500 samples is usually 10 seconds
input_shape = (num_channels, 2500)


In [84]:
model = models.Sequential()
model.add(layers.Conv1D(16, 5, activation = 'relu', input_shape = input_shape))
model.add(layers.Conv1D(32, 5, activation = 'relu', input_shape = input_shape))
model.add(layers.MaxPool1D(
    pool_size=2,
    strides=None,
    padding='valid',))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Model Training


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))